In [1]:
minmaxInterval = 2000

## open port

In [2]:
!echo 'quzinee1!'| sudo -S chmod a+rw /dev/ttyUSB1

[sudo] password for terry: chmod: cannot access '/dev/ttyUSB1': No such file or directory


## BASIC USAGE OF XL320 CLASS

In [3]:
import os, time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from math import pi as PI
DEG90 = PI/2
PROPORTION_MATRIX = np.array(
    [
        [1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.1,1,0.4,0.4, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0,  1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.1,1,0.4,0.4,  0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1],
        ])
PROPORTION_MATRIX = PROPORTION_MATRIX[:4,:4]
PROPORTION_MATRIX.shape

(4, 4)

In [4]:
import sys
sys.path.append(".")
""" FOR DYNAMIXEL """
from model.class_xc330 import xc330
""" FOR MODERN DRIVER """
import roslib; roslib.load_manifest('ur_driver')
import rospy
import actionlib
from control_msgs.msg import *
from trajectory_msgs.msg import *
from sensor_msgs.msg import JointState

from class_optitrack_base import OptiTrackBase
from class_optitrack_marker import OptiTrack

class OneFinger(object):
    def __init__(self,):
        rospy.init_node("DEMO")
        self.JOINT_NAMES = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
                            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
        self.client      = None
        
        self.xc          = xc330('SOFTROBOT', _USB_NUM=0)
        
    def init_xc(self,initpos):
        self.xc.connect()
        self.xc.set_operatingmode([4])

        self.xc.set_torque([0])
        time.sleep(2)
        self.xc.set_torque([1])
        print ("TORQUE ON")

        currpos = self.xc.get_currpos()
        minmaxInterval = 5000
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)

        if initpos is None:
            initpos = currpos; print("No initpos Found")
            self.set_init_pos()
            return currpos

        self.xc.set_goalposcluster(initpos,1)

        print("INITIALIZE POSITION")
        currpos = self.xc.get_currpos()
        print(currpos)

        minmaxInterval = minmaxInterval
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)
        self.set_init_pos()
        return currpos
        
    def set_init_pos(self):
        self.initpos = self.xc.get_currpos()
    # def move_xc(self,pos):
    #     pos = self.initpos + np.array(pos)* np.array([1,-1,-1,1])
    #     self.xc.set_goalposcluster(pos,1)
        
        
    def move_arm_speed(self, joints, speed):
        try: 
            q = joints
            g = FollowJointTrajectoryGoal()
            g.trajectory = JointTrajectory()
            g.trajectory.joint_names = self.JOINT_NAMES
            joint_states = rospy.wait_for_message("joint_states", JointState)
            joints_pos   = joint_states.position

            diff = np.array(joints_pos) - np.array(q)

            assert speed <= 1e-1 
            movetime = np.linalg.norm(diff)/speed

            g.trajectory.points = [
                JointTrajectoryPoint(positions=joints_pos, velocities=[0]*6, time_from_start=rospy.Duration(0.0)),
                JointTrajectoryPoint(positions=q, velocities=[0]*6, time_from_start=rospy.Duration(movetime))]  

            self.client.send_goal(g)
            self.client.wait_for_result()
        except KeyboardInterrupt:
            self.client.cancel_goal()
            raise
        except:
            raise  
        
    def move_xc(self, control, movetime):
        controls = np.array(control).astype(int)
        assert controls.ndim == 1

        
        pos_list = []
        currpos = self.xc.get_currpos()
        goal_pos  = control * np.array([1,-1,-1,1]) + self.initpos

        resolution = 5
        splitnum = np.linalg.norm(goal_pos - currpos)/resolution

        interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
        currpos = goal_pos
        pos_list.append(interpolate_pos)
        
        pos_array = np.vstack(pos_list)

        if len(pos_array) > 0:
            unit_sleep = movetime/len(pos_array)
            for pos in pos_array:
                self.xc.set_goalpos(pos)
                time.sleep(unit_sleep)


    def multiple_move_xc(self, controls, movetimes):
        controls = np.array(controls).astype(int)
        movetimes = np.array(movetimes)
        assert controls.ndim == 2
        assert movetimes.ndim == 1
        
        pos_list = []
        currpos = self.xc.get_currpos()

        for control in controls:
            goal_pos  = control * np.array([1,-1,-1,1]) + self.initpos

            resolution = 5
            splitnum = np.linalg.norm(goal_pos - currpos)/resolution

            interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
            currpos = goal_pos
            pos_list.append(interpolate_pos)
            
        # pos_array = np.vstack(pos_list)
        for pos, movetime in zip(pos_list, movetimes):
            unit_sleep = movetime/len(pos)
            for pos_ in pos:
                self.xc.set_goalpos(pos_)
                time.sleep(unit_sleep)

    def execute_arm_speed(self, joints, speed):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm_speed(joints, speed)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      


['/home/terry/Rilab/sOftrobot/SORO-Dynamixel-python', '', '/home/terry/catkin_ws/devel/lib/python3/dist-packages', '/opt/ros/noetic/lib/python3/dist-packages', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '/home/terry/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '.']


## INSTANTIATE Robot 

In [5]:
robot = OneFinger()

[SOFTROBOT] INSTANTIATED AT [b'/dev/ttyUSB0']


In [6]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90, 0]

In [7]:
currpos = robot.init_xc([4722, 3082,  957, 1628])

[SOFTROBOT] CONNECT
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [4]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[13, 14, 15, 16]]
TORQUE ON
INITIALIZE POSITION
[4686 3120 1001 1614]


In [11]:
robot.move_xc([0,0,0,1000], movetime=5)

In [9]:
def move_soro(robot:OneFinger, motor_array, time_array):
    motor_array = np.array(motor_array)

    assert motor_array.ndim == 2
    assert len(time_array) == len(motor_array)

    motor_list = []
    currpos = robot.xc.get_currpos()
    
    for motor_control in motor_array:
        goal_pos  = PROPORTION_MATRIX @ motor_control * np.array([1,-1,-1,1]) + robot.initpos

        resolution = 10
        splitnum = np.linalg.norm(goal_pos - currpos)/resolution

        interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
        currpos = goal_pos
        motor_list.append(interpolate_pos)
    
    from tqdm.notebook import tqdm
    for motor_array, movetime in tqdm(zip(motor_list, time_array)):
        
        unit_sleep = movetime/len(motor_array)
        for motor in motor_array:
            robot.xc.set_goalpos(motor)
            time.sleep(unit_sleep* 0.7)
        




In [10]:
motor_array = np.array([[0,0,0,0]]).astype(np.float32)
time_array= np.array([1.0])

move_soro(robot, motor_array, time_array)

0it [00:00, ?it/s]

/tmp/ipykernel_38315/2316108711.py:23: RuntimeWarning: divide by zero encountered in double_scalars
  unit_sleep = movetime/len(motor_array)


In [11]:
class Mocap():
    def __init__(self):
        self.JOINT_NAMES = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
                            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
        self.client      = None
        self.mocap_marker= OptiTrack()
        self.mocap_platform = OptiTrackBase('Platform')
        
    

In [12]:
import json

mocap = Mocap()

In [13]:
import json

def get_mocap_info(mocap:Mocap):
    integrated_dict = {}

    platform_data_ = mocap.mocap_platform.transform_data
    platform_dict_ = dict(
        plat_x=platform_data_.position.x,
        plat_y=platform_data_.position.y,
        plat_z=platform_data_.position.z)
    integrated_dict.update(platform_dict_)
    

    markers = mocap.mocap_marker.markers
    assert len(markers) == 1
    marker = markers[0]
    EE_dict_ = dict(
        EE_x = marker.position.x,
        EE_y = marker.position.y,
        EE_z = marker.position.z
    )

    integrated_dict.update(EE_dict_)
    
    return integrated_dict

mocap_info = get_mocap_info(mocap)

In [14]:
p_plat = np.array([mocap_info['plat_x'], mocap_info['plat_y'], mocap_info['plat_z']])
p_ee = np.array([mocap_info['EE_x'], mocap_info['EE_y'], mocap_info['EE_z']])

p_zero = p_ee-p_plat
p_zero

array([-0.00612204,  0.12711757,  0.00408193])

In [15]:
def get_random_motor_control(limit=2000):
    r = np.random.rand(1).astype(np.float32) * limit
    theta = np.random.rand(1).astype(np.float32) * 2 * np.pi

    motor_control = np.array([r*np.cos(theta), r*np.sin(theta)]).astype(np.float32)

    return motor_control

def motor_control_to_motor_array(motor_control):
    motor_array = np.zeros(4).astype(np.float32)

    # motor_array[1] = motor_control[0]
    # motor_array[2] = motor_control[1]

    if motor_control[0] > 0:
        motor_array[0] = motor_control[0]
    else:
        motor_array[1] = -motor_control[0]

    if motor_control[1] > 0:
        motor_array[2] = motor_control[1]
    else:
        motor_array[3] = -motor_control[1]

    motor_array = motor_array.reshape(1,-1) 
    
    return motor_array


motor_control = get_random_motor_control(limit=2000)
motor_control_to_motor_array(motor_control), motor_control

(array([[163.13475,   0.     ,   0.     , 660.543  ]], dtype=float32),
 array([[ 163.13475],
        [-660.543  ]], dtype=float32))

# Repeat Test

In [16]:
'''
flag = False
error_list = []
for motor_array in [[1000,0,0,0],[0,1000,0,0],[0,0,1000,0],[0,0,0,1000]]:
    target_motor = np.array(motor_array).reshape(1,-1)
    position_list = []
    for i in range(10):
        try:
            motor_array = np.zeros((1,4)).astype(np.float32)
            time_array= np.array([1.0])    
            move_soro(robot, motor_array, time_array)   

            motor_array = target_motor
            time_array= np.array([1.0])
            move_soro(robot, motor_array, time_array)   

            time.sleep(1.0)

            mocap_info = get_mocap_info(mocap)


            position = np.array([mocap_info['EE_x'], mocap_info['EE_y'], mocap_info['EE_z']])
            position_list.append(position)

        
        except KeyboardInterrupt:
            break

        except Exception as exception:

            print(f'error in motor_array:{motor_array}', end='\r')
            
            
            print("Hard Escaping the error") 
            print(exception)
            flag = True
            break

    if flag: break
    position_array = np.array(position_list)
    error = np.std(position_array, axis=0)
    error_list.append(error)

error_list
'''

'\nflag = False\nerror_list = []\nfor motor_array in [[1000,0,0,0],[0,1000,0,0],[0,0,1000,0],[0,0,0,1000]]:\n    target_motor = np.array(motor_array).reshape(1,-1)\n    position_list = []\n    for i in range(10):\n        try:\n            motor_array = np.zeros((1,4)).astype(np.float32)\n            time_array= np.array([1.0])    \n            move_soro(robot, motor_array, time_array)   \n\n            motor_array = target_motor\n            time_array= np.array([1.0])\n            move_soro(robot, motor_array, time_array)   \n\n            time.sleep(1.0)\n\n            mocap_info = get_mocap_info(mocap)\n\n\n            position = np.array([mocap_info[\'EE_x\'], mocap_info[\'EE_y\'], mocap_info[\'EE_z\']])\n            position_list.append(position)\n\n        \n        except KeyboardInterrupt:\n            break\n\n        except Exception as exception:\n\n            print(f\'error in motor_array:{motor_array}\', end=\'\r\')\n            \n            \n            print("Hard Es

In [20]:
# error_list

NameError: name 'error_list' is not defined

# Collection

In [17]:
from pathlib import Path
datapath = Path('data')

ls = [0]
for path_ in datapath.iterdir():
    num = str.split(str(path_.stem),"_")[-1]
    ls.append(int(num))

start_episode = max(ls)+1
start_episode

842

In [19]:
from pathlib import Path
datapath = Path('data')

ls = [0]
for path_ in datapath.iterdir():
    num = str.split(str(path_.stem),"_")[-1]
    ls.append(int(num))

start_episode = max(ls)+1
num = start_episode
error_num = 0

while num < start_episode+500:
    try:
        motor_array = np.zeros((1,4)).astype(np.float32)
        time_array= np.array([1.0])    
        move_soro(robot, motor_array, time_array)   

        motor_control = get_random_motor_control(limit=2000)
        motor_array = motor_control_to_motor_array(motor_control)
        time_array= np.array([1.0])

        move_soro(robot, motor_array, time_array)   

        time.sleep(1.0)

        mocap_info = get_mocap_info(mocap)

        episode_info = {}
        episode_info.update(mocap_info)
        episode_info['p_zero'] = p_zero.tolist()
        episode_info.update(dict(motor_control=motor_control.tolist()))
        json.dump(episode_info, open(f'data/episode_info_{num}.json','w'))
        num += 1
    
        error_num = 0
    
    except KeyboardInterrupt:
        break

    except:
        print(f'error in motor_array:{motor_array}')
        # break
        # error_num = error_num +1
        # if error_num == 10: break

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]